# Word2vec: работаем с векторными моделями в Python

*Эта тетрадка — переосмысленная, дополненная и местами упрощенная мной версия туториала по вордтувеку от Лизы Кузьменко, которая со-основала RusVectores вместе с Андреем Кутузовым*


**Word2vec** - библиотека для получения векторных представлений слов на основе их совместной встречаемости в текстах. Вы можете освежить в памяти механизмы работы **word2vec**, прочитав [эту статью](https://vk.com/@sysblok-word2vec-pokazhi-mne-svoi-kontekst-i-ya-skazhu-kto-ty). 

Сейчас мы научимся использовать **word2vec** в своей повседневной работе. Мы будем использовать реализацию **word2vec** в библиотеке [gensim](https://radimrehurek.com/gensim/) для языка программирования **python**.

Для работы с эмбеддингами слов существуют и другие библиотеки: кроме [gensim](https://radimrehurek.com/gensim/) можно делать векторные модели в [keras](https://keras.io/), [tensorflow](https://www.tensorflow.org/), [pytorch](https://pytorch.org/). Но мы будем работать с *gensim*, потому что тут это проще и потому что создатели библиотеки моделей RusVectores затачивались под нее.


***Gensim***  - изначально библиотека для тематического моделирования текстов. Однако помимо различных алгоритмов для *topic modeling* в ней реализованы на **python** и алгоритмы из тулкита **word2vec** (который в оригинале был написан на C++). Если вы работаете на своей машине и **gensim** у вас не установлен, нужно его установить: `pip3 install gensim`

В колабе генсим установлен по умолчанию


In [1]:
import gensim

Тьюториал состоит из двух частей:
* В первой части мы разберёмся, как загружать уже готовые векторные модели и работать с ними. Например, мы научимся выполнять простые операции над векторами слов, такие как «найти слово с наиболее близким вектором» или «вычислить коэффициент близости между двумя векторами слов». Также мы рассмотрим более сложные операции над векторами, например, «вычесть из вектора слова вектор другого слова», «прибавить к вектору слова вектор другого слова»  «найти лишний вектор в группе слов».

* Во второй части мы научимся предобрабатывать текстовые файлы и самостоятельно тренировать векторную модель на своих данных.

## 1. Работа с готовыми векторными моделями при помощи библиотеки Gensim

Для своих индивидуальных нужд и экспериментов бывает полезно самому натренировать модель на нужных данных и с нужными параметрами (об этом раздел 2). Но для каких-то общих целей уже есть готовые модели, в т.ч. для русского языка, обученные на больших корпусах

Модели для русского скачать можно здесь - https://rusvectores.org/ru/models/

Существуют несколько форматов, в которых могут храниться модели. Во-первых, данные могут храниться в нативном формате *word2vec*, при этом модель может быть бинарной или не бинарной. Для загрузки модели в формате *word2vec* в классе `KeyedVectors` (в котором хранится большинство относящихся к дистрибутивным моделям функций) существует функция `load_word2vec_format`, а бинарность модели можно указать в аргументе `binary` (внизу будет пример). Помимо этого, модель можно хранить и в собственном формате *gensim*, для этого существует класс `Word2Vec` с функцией `load`. Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте. Наш код определяет тип модели по её расширению, но вообще файл с моделью может называться как угодно, жестких ограничений для расширения нет.

Давайте скачаем новейшую модель для русского языка, созданную на основе [Национального корпуса русского языка (НКРЯ)](http://www.ruscorpora.ru/) (поскольку zip-архив с моделью весит почти 500 мегабайт, следующая ячейка выполнится у вас не сразу!). 


In [2]:
!wget 'http://vectors.nlpl.eu/repository/20/180.zip'

--2022-03-16 20:17:22--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘180.zip’

180.zip             100%[===================>] 462.01M  5.37MB/s    in 70s     

2022-03-16 20:18:32 (6.59 MB/s) - ‘180.zip’ saved [484452317/484452317]



Теперь моделька в виде zip-архива лежит у нас в рабочей папке

In [3]:
!ls

180.zip
class1.ipynb
spam.csv
vector_semantics_1_(w2v,_fasttext).ipynb


Распаковывать скачанный архив для обычных моделей не нужно, так как его содержимое прочитается при помощи специальной инструкции:

In [4]:
import zipfile

In [5]:
with zipfile.ZipFile('180.zip', 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [6]:
type(model)

gensim.models.keyedvectors.KeyedVectors

### Все, этой моделью уже можно пользоваться для оценки семантической близости:

Выводим 10 соседей слова по близости и меру близости с ними — метод `most_similar`:

In [18]:
model.most_similar ('вишня_NOUN')

[('яблоня_NOUN', 0.6901825070381165),
 ('слива_NOUN', 0.6638981699943542),
 ('черешня_NOUN', 0.6636965870857239),
 ('абрикос_NOUN', 0.65395587682724),
 ('груша_NOUN', 0.6477444767951965),
 ('яблоко_NOUN', 0.6413966417312622),
 ('персик_NOUN', 0.6345474123954773),
 ('вишня_VERB', 0.63399738073349),
 ('алыча_NOUN', 0.6283605694770813),
 ('слива_ADJ', 0.612352728843689)]

⛳ 💻  Вопрос: как получить больше 10?

In [21]:
## ваш код
model.most_similar ('лингвист_NOUN', topn=5)

[('языковед_NOUN', 0.7750651836395264),
 ('филолог_NOUN', 0.6346759796142578),
 ('лингвистика_NOUN', 0.629889190196991),
 ('антрополог_NOUN', 0.6117334365844727),
 ('лингвистический_ADJ', 0.607776939868927)]

In [22]:
model.most_similar ('программирование_NOUN')

[('java_PROPN', 0.6837120652198792),
 ('алгоритмический_ADJ', 0.6621100306510925),
 ('интерфейс_NOUN', 0.6615059971809387),
 ('-технология_NOUN', 0.6228690147399902),
 ('xml_PROPN', 0.6216764450073242),
 ('html_PROPN', 0.6212592124938965),
 ('информатика_NOUN', 0.6158244013786316),
 ('пользовательский_ADJ', 0.6138737797737122),
 ('формализованный_ADJ', 0.6094743609428406),
 ('формализация_NOUN', 0.6024623513221741)]

In [23]:
model.most_similar ('программировать_VERB')

[('моделировать_VERB', 0.6391699910163879),
 ('запрограммированный_VERB', 0.6083677411079407),
 ('смоделировать_VERB', 0.5745291113853455),
 ('запрограммировать_VERB', 0.5709876418113708),
 ('просчитывать_VERB', 0.5686773657798767),
 ('задывать_VERB', 0.5433831810951233),
 ('детерминированный_VERB', 0.5256835222244263),
 ('предопределять_VERB', 0.5215480327606201),
 ('детерминировать_VERB', 0.5210214257240295),
 ('срабатывать_VERB', 0.509833574295044)]

### 🤔 стоп, а что за _NOUN, _VERB это мне всегда руками так писать?

Cлова в модель надо подавать с указанием части речи (pos tag) из набора тегов [Universal POS-tags](https://universaldependencies.org/u/pos/). Так устроены модели RusVectores — в них снята омонимия на уровне словоформ:

In [24]:
model.most_similar ('печь_NOUN')

[('печка_NOUN', 0.8553919792175293),
 ('печурка_NOUN', 0.6798753142356873),
 ('печной_ADJ', 0.6434558033943176),
 ('лежанка_NOUN', 0.6250245571136475),
 ('топка_NOUN', 0.6232640743255615),
 ('дымоход_NOUN', 0.6071035861968994),
 ('печи_NOUN', 0.602951169013977),
 ('чугун_NOUN', 0.5874652862548828),
 ('полать_NOUN', 0.5816735029220581),
 ('буржуйка_NOUN', 0.5790904760360718)]

In [25]:
model.most_similar ('печь_VERB')

[('испекать_VERB', 0.6701913475990295),
 ('выпекать_VERB', 0.6453807353973389),
 ('жарить_VERB', 0.622571587562561),
 ('жариться_VERB', 0.6005048155784607),
 ('напечь_VERB', 0.5875173211097717),
 ('стряпать_VERB', 0.5786464810371399),
 ('пек_NOUN', 0.5724241733551025),
 ('оладья_NOUN', 0.5647705793380737),
 ('запекать_VERB', 0.5645115971565247),
 ('варить_VERB', 0.561690092086792)]

Давайте пока работаем с небольшим числом слов писать это руками. А дальше поговорим, как с этим работать автоматически, если анализируем большой текст. 

Если мы прогоняем много слов, стоит вставить проверку, что слова нет в модели. Допустим, нам интересны такие слова (пример для русского языка):

In [26]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'студент_ADJ']

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [27]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

день_NOUN
неделя_NOUN 0.7375996112823486
день_PROPN 0.706766664981842
месяц_NOUN 0.7037326097488403
час_NOUN 0.6643950939178467
утро_NOUN 0.6526744961738586
вечер_NOUN 0.6038411259651184
сутки_NOUN 0.5923080444335938
воскресенье_NOUN 0.5842781066894531
полдень_NOUN 0.5743688344955444
суббота_NOUN 0.5345946550369263


ночь_NOUN
ночь_PROPN 0.8310787081718445
вечер_NOUN 0.7183678150177002
рассвет_NOUN 0.6965947151184082
ночи_NOUN 0.692021906375885
полночь_NOUN 0.6704976558685303
ночь_VERB 0.6615265011787415
утро_NOUN 0.6263936161994934
ночной_ADJ 0.6024709343910217
полдень_NOUN 0.5835085511207581
сумерки_NOUN 0.5671443939208984


человек_NOUN
человек_PROPN 0.7850059270858765
человеческий_ADJ 0.5915265679359436
существо_NOUN 0.5736929774284363
народ_NOUN 0.5354466438293457
личность_NOUN 0.5296981930732727
человечество_NOUN 0.5282931327819824
человкъ_PROPN 0.5047001242637634
индивидуум_NOUN 0.5000404119491577
нравственный_ADJ 0.4972919225692749
потому_ADV 0.49293625354766846


семантика_NOU

Наш код сказал нам, что прилагательного студент модель не знает...

In [30]:
"сани_NOUN" in model

True

### сравнить близость 2 слов:

Находим косинусную близость пары векторов слов — метод `similarity`:

In [33]:
print(model.similarity('красный_ADJ', 'синий_ADJ'))
print(model.similarity('красный_ADJ', 'зеленый_ADJ'))

0.6052179
0.5960778


In [34]:
print(model.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.22025342


In [31]:
print(model.similarity('человек_NOUN', 'женщина_NOUN'))
print(model.similarity('человек_NOUN', 'мужчина_NOUN'))

0.4803466
0.36275825


In [39]:
print(model.similarity('мужчина_NOUN', 'женщина_NOUN'))
print(model.similarity('король_NOUN', 'королева_NOUN'))
print(model.similarity('принц_NOUN', 'принцесса_NOUN'))

0.7777514
0.7041438
0.7602008


In [35]:
print(model.similarity('человек_NOUN', 'картофель_NOUN'))

0.030121876


<img src="https://www.meme-arsenal.com/memes/32687f97c291d55dc3143e26821ff4d4.jpg">

Мистер картошка, вы раскрыты! 

## 2. Более сложные операции над векторами

Помимо более простых операций над векторами (нахождение косинусной близости между двумя векторами и ближайших соседей вектора) **gensim** позволяет выполнять и более сложные операции над несколькими векторами. Так, например, мы можем найти лишнее слово в группе. Лишним словом является то, вектор которого наиболее удален от других векторов слов.

In [36]:
words = ['яблоко_NOUN', 'груша_NOUN', 'виноград_NOUN', 'банан_NOUN', 'лимон_NOUN', 'картофель_NOUN', 'лошадь_NOUN', 'философия_NOUN', 'видеоблог_NOUN']

In [37]:
print(model.doesnt_match(words))

философия_NOUN


Также можно складывать и вычитать вектора нескольких слов. Например, сложив два вектора и вычтя из них третий вектор, мы можем решить своеобразную пропорцию. Подробнее о семантических пропорциях вы можете прочитать в [материале Системного Блока](https://vk.com/@sysblok-vo-chto-prevraschaetsya-zhizn-bez-lubvi).

Меня всегда радует, что вот это реально работает: 
🦅 -> 🐠

In [40]:
print(model.most_similar(positive=['рыба_NOUN', 'крыло_NOUN'], negative=['плавник_NOUN'])[0][0])

птица_NOUN


В обратную сторону тоже: 🐠 -> 🦅 

In [41]:
print(model.most_similar(positive=['птица_NOUN', 'плавник_NOUN'], negative=['крыло_NOUN'])[0][0])

рыба_NOUN


Ну и конечно: 💔

In [42]:
print(model.most_similar(positive=['жизнь_NOUN'], negative=['любовь_NOUN'])[0][0])

быт_NOUN


Ну и напоследок: шит хитз зе фэн 💩💩💩

In [43]:
print(model.most_similar(positive=['ссср_PROPN', 'гитлер_PROPN'], negative=['германия_PROPN'])[0][0])

сталин_PROPN


In [47]:
print(model.most_similar(positive=['король_NOUN', 'женщина_NOUN'], negative=['мужчина_NOUN'], topn=5))

[('герцог_NOUN', 0.6892667412757874), ('король_PROPN', 0.6768986582756042), ('королева_NOUN', 0.6501403450965881), ('королева_ADV', 0.6257951855659485), ('дофин_NOUN', 0.6146250367164612)]


In [49]:
print(model.most_similar(positive=['программист_NOUN'], negative=['клавиатура_NOUN'])[0][0])

юрист_NOUN


In [68]:
print(model.most_similar(positive=['время_NOUN', 'деньги_NOUN'], topn=5))

[('деньги_PROPN', 0.6096085906028748), ('сбережения_NOUN', 0.5187363028526306), ('денеод_NOUN', 0.5150483250617981), ('сумма_NOUN', 0.5042823553085327), ('средства_NOUN', 0.488772451877594)]


### ⛳ 💻  Задание: подберите еще 3-4 симпатичных примера векторной арифметики

## 3. Предобработка текстовых данных

Вернемся к вопросу о pos-тегах (_NOUN, _VERB и проч). Пока мы обходились без них, но это выглядело как костыль, правда же 🔩 Если мы хотим обрабатывать свои тексты — нам надо бы научиться предобрабатывать их так, чтобы каждое слово шло именно с таким тегом. Тогда можно будет гонять на них модели word2vec от RusVectores (а это лучшее что есть для русского и вообще такой стандартный стандарт).  

Предобработка текстов для тренировки моделей выглядит следующим образом:
* сначала мы приведем все слова к начальной форме (лемматизируем) и удалим стоп-слова;
* затем мы приведем все леммы к нижнему регистру;
* для каждого слова добавим его частеречный тэг.

Давайте попробуем воссоздать процесс предобработки текста на примере [сказки Хармса](https://raw.githubusercontent.com/dhhse/dh2020/master/data/harms.txt). Для предобработки можно использовать различные тэггеры, мы сейчас будем использовать [*UDPipe*](https://ufal.mff.cuni.cz/udpipe), чтобы сразу получить частеречную разметку в виде Universal POS-tags. Сначала установим обертку *UDPipe* для Python:

In [69]:
!pip install ufal.udpipe

You should consider upgrading via the '/Users/Alexey.Zhebel/IdeaProjects/CompLing/venv/bin/python -m pip install --upgrade pip' command.


*UDPipe* использует предобученные модели для лемматизации и тэггинга. Вы можете использовать [уже готовую модель](https://rusvectores.org/static/models/udpipe_syntagrus.model) или обучить свою. 

Кусок кода ниже скачает модель UDPipe для лингвистической предобработки. Модель весит 40 мегабайт, поэтому ячейка может выполнятся некоторое время, особенно если у вас небыстрый интернет. 

In [70]:
!wget 'https://rusvectores.org/static/models/udpipe_syntagrus.model'

--2022-03-16 21:10:20--  https://rusvectores.org/static/models/udpipe_syntagrus.model
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40616122 (39M)
Saving to: ‘udpipe_syntagrus.model’

udpipe_syntagrus.mo 100%[===================>]  38.73M  81.3KB/s    in 10m 12s 

2022-03-16 21:20:32 (64.8 KB/s) - ‘udpipe_syntagrus.model’ saved [40616122/40616122]



In [71]:
modelfile = 'udpipe_syntagrus.model'

## ⚠️ Для соответствия моделям RusVectores требуется еще немножко допиливания напильником поверх UDPipe. Да и сама машинерия UDPipe довольно громоздко устроена (там выдача [в формате CONLLU](https://universaldependencies.org/format.html)). Поэтому ниже я просто переиспользую функции, которые написала со-авторка RusVectores Лиза Кузьменко для предобработки при помощи UDPipe и использую их. Но в целом там вроде бы ничего магического не происходит:

Приступим к собственно предобработке текста. Попробуем лемматизировать текст и добавить частеречные тэги при помощи этой функции:

In [72]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn


Эту функцию можно также изменить под конкретную задачу. Например, если частеречные тэги нам не нужны, в функции ниже выставим `keep_pos=False`. Если необходимо сохранить знаки пунктуации, можно выставить `keep_punct=True`. 

Теперь загружаем модель *UDPipe*, читаем текстовый файл и обрабатываем его при помощи нашей функции. В файле должен содержаться необработанный текст (одно предложение на строку или один абзац на строку).
Этот текст токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход мы получаем последовательность разделенных пробелами лемм с частями речи ("зеленый\_NOUN трамвай\_NOUN").

In [73]:
# в функции ниже используется питоновский модуль wget (не то же самое, что !wget выше)
# на тот случай, если модель не скачана -- он ее автоматически перескачает
# поэтому в этой ячейке установим питоновский wget
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9673 sha256=0210c80742a844c17a36e5760db5249181e9f8b3b049df48f1c2c25514f7bf09
  Stored in directory: /Users/Alexey.Zhebel/Library/Caches/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
You should consider upgrading via the '/Users/Alexey.Zhebel/IdeaProjects/CompLing/venv/bin/python -m pip install --upgrade pip' command.


In [74]:
from ufal.udpipe import Model, Pipeline
import os
import re
import sys
import wget

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    lines = text.split('\n')
    tagged = []
    for line in lines:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        tagged_line = ' '.join(output)
        tagged.append(tagged_line)
    return '\n'.join(tagged)

def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken

Скачаем теперь текст, с которым будем работать:

In [75]:
!wget 'https://raw.githubusercontent.com/dhhse/dh2020/master/data/harms.txt'

--2022-03-16 21:42:02--  https://raw.githubusercontent.com/dhhse/dh2020/master/data/harms.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8147 (8.0K) [text/plain]
Saving to: ‘harms.txt’

harms.txt           100%[===================>]   7.96K  --.-KB/s    in 0s      

2022-03-16 21:42:02 (28.8 MB/s) - ‘harms.txt’ saved [8147/8147]



In [77]:
skazka = open('harms.txt', 'r', encoding='utf-8').read()
processed_text = tag_ud(text=skazka, modelfile=modelfile)
print(processed_text[:350])
with open('harms_processed.txt', 'w', encoding='utf-8') as out:
    out.write(processed_text)


Loading the model...
Processing input...


Сказка_NOUN
вот_PART сказать_VERB Ваня_PROPN  класть_VERB на_ADP стол_NOUN тетрадка_NOUN давать_VERB писать_VERB сказка_NOUN
давать_VERB сказать_VERB Леночка_PROPN  садиться_VERB на_ADP стул_NOUN
Ваня_PROPN  брать_VERB карандаш_NOUN и_CCONJ писать_VERB
Жил-был_VERB король_NOUN
тут_ADV Ваня_PROPN  задумываться_VERB и_CCONJ поднимать_VERB глаз_NOUN к


Наша функция напечатает обработанный текст, который мы теперь можем также сохранить в файл. 

Итак, в ходе этой части тьюториала мы научились от "сырого текста" приходить к лемматизированному тексту с частеречными тэгами, который уже можно подавать на вход модели! 




In [79]:
with open('harms_processed.txt', 'r', encoding='utf-8') as tagged_text:
    words = tagged_text.read().split()
    for word in words[:20]:
      print ('слово: ', word)
      if word in model:
        x = model.most_similar(word)
        print ('ближайший синоним: ', x[0][0])

слово:  Сказка_NOUN
слово:  вот_PART
слово:  сказать_VERB
ближайший синоним:  говорить_VERB
слово:  Ваня_PROPN
слово:  класть_VERB
ближайший синоним:  положить_VERB
слово:  на_ADP
слово:  стол_NOUN
ближайший синоним:  столик_NOUN
слово:  тетрадка_NOUN
ближайший синоним:  тетрадь_NOUN
слово:  давать_VERB
ближайший синоним:  дать_VERB
слово:  писать_VERB
ближайший синоним:  написать_VERB
слово:  сказка_NOUN
ближайший синоним:  сказка_PROPN
слово:  давать_VERB
ближайший синоним:  дать_VERB
слово:  сказать_VERB
ближайший синоним:  говорить_VERB
слово:  Леночка_PROPN
слово:  садиться_VERB
ближайший синоним:  усаживаться_VERB
слово:  на_ADP
слово:  стул_NOUN
ближайший синоним:  табурет_NOUN
слово:  Ваня_PROPN
слово:  брать_VERB
ближайший синоним:  взять_VERB
слово:  карандаш_NOUN
ближайший синоним:  карандашик_NOUN


[Вернемся к слайдам ненадолго](https://docs.google.com/presentation/d/11fYkNG1IFBJzVQ27LNxaE_fj8XHO40JxYaUUSzpcCKQ/edit#slide=id.gd5da728dca_0_1009) — нам осталась fun part сегодняшней пары! 🎪

⛳ 💻  Я предлагаю вам реализовать свою версию  [векторных романов Б. Орехова](https://habr.com/ru/post/326380/). В базовом варианте предлагаю делать упрощенно: без восстановления морфологической формы после замены слова на его векторный синоним (можем для корректности назвать это семантическим ассоциатом или квази-синонимом). Ну то есть заменяем "бегемотом" на "гиппопотам" (или что там выдаст word2vec), но форму "гиппопотамом" уже не восстанавливаем. 

А кто чувствует в себе силы — делайте полную версию, с восстановлением исходной грамматической формы "гиппопотамом". За это оценка будет выше. Ну и результат у вас будет гораздо прикольнее, потому что текст будет довольно читабелен.

К <b><s>10</s> 16 мая</b> нужно сдать любое крупное произведение русской литературы, в котором слова заменены на их векторные семантические ассоциаты. Ну и, конечно, код, который это делает. Код из этой тетрадки можно переиспользовать.

## 4. Тренируем свою модель в gensim (остается на домашние эксперименты и следующий раз)

Поскольку обучение и загрузка моделей могут занимать продолжительное время, иногда бывает полезно вести лог событий. Для этого используется стандартная питоновская библиотека **logging**.

In [80]:
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

На вход модели мы даем наш обработанный текстовый файл (либо любой другой текст, важно лишь, что каждое предложение должно быть на отдельной строчке).

In [81]:
f = 'harms_processed.txt'
data = gensim.models.word2vec.LineSentence(f)

Инициализируем модель. Параметры в скобочках:
* data - данные, 
* size - размер вектора, 
* window - размер окна наблюдения,
* min_count - мин. частотность слова в корпусе, которое мы берем,
* sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

In [83]:
model = gensim.models.Word2Vec(data, vector_size=500, window=10, min_count=2, sg=0)

2022-03-16 22:07:09,980 : INFO : collecting all words and their counts
2022-03-16 22:07:09,981 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-16 22:07:09,982 : INFO : collected 270 word types from a corpus of 690 raw words and 39 sentences
2022-03-16 22:07:09,983 : INFO : Creating a fresh vocabulary
2022-03-16 22:07:09,984 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 retains 111 unique words (41.111111111111114%% of original 270, drops 159)', 'datetime': '2022-03-16T22:07:09.984207', 'gensim': '4.1.2', 'python': '3.8.9 (default, Oct 26 2021, 07:25:54) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.1-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-03-16 22:07:09,984 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 leaves 531 word corpus (76.95652173913044%% of original 690, drops 159)', 'datetime': '2022-03-16T22:07:09.984866', 'gensim': '4.1.2', 'python': '3.8.9 (default, Oct 26 2021, 07:25:54) \n

In [ ]:
# вариант для новой версии gensim:
# model = gensim.models.Word2Vec(data, vector_size=500, window=10, min_count=2, sg=0)

Мы создаем модель, в которой размерность векторов — 500, размер окна наблюдения — 10 слов, алгоритм обучения — CBOW, слова, встретившиеся в корпусе только 1 раз, не используются. После тренировки модели можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировать.

In [84]:
model.init_sims(replace=True)

/var/folders/b6/jlv0gzhn72vdj8t4yttnz9cw0000kt/T/ipykernel_15631/3625671219.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)
2022-03-16 22:20:41,591 : WARNING : destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


Смотрим, сколько в модели слов:

In [89]:
print(len(model.wv.vocab))

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [90]:
print(len(model.wv.key_to_index))

111


И сохраняем!

In [91]:
model.save('my.model')

2022-03-16 22:23:17,042 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'my.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-16T22:23:17.042630', 'gensim': '4.1.2', 'python': '3.8.9 (default, Oct 26 2021, 07:25:54) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.1-x86_64-i386-64bit', 'event': 'saving'}
2022-03-16 22:23:17,043 : INFO : not storing attribute cum_table
2022-03-16 22:23:17,045 : INFO : saved my.model


In [95]:
model.most_similar('нога_NOUN')

AttributeError: 'Word2Vec' object has no attribute 'most_similar'

In [96]:
# для новой версии генсим, где не работает просто most_similar:
model.wv.most_similar ('карандаш_NOUN')

[('и_CCONJ', 0.14096979796886444),
 ('комната_NOUN', 0.11021428555250168),
 ('1-е_NOUN', 0.10871057212352753),
 ('о_ADP', 0.10352557897567749),
 ('писать_VERB', 0.1028280258178711),
 ('падать_VERB', 0.09768801927566528),
 ('тетрадка_NOUN', 0.09694117307662964),
 ('кузнеце_NOUN', 0.09089557826519012),
 ('кулак_NOUN', 0.08956850320100784),
 ('там_ADV', 0.08054307848215103)]

### ⛳ 💻 Задание

Задание: попробуйте обучить свою модель на каких-нибудь текстах. Например, [вот текст "Войны и мира"](https://github.com/dhhse/dh2020/blob/master/data/wap_w2v.txt), в котором каждое предложение с новой строки. Обучите модель на нем. Исследуйте близости слов. 

*   Подсказка: вам точно понадобится `gensim.models.word2vec.LineSentence` (см. выше) чтобы преобразовать текст в формат, который можно передавать для обучения модели
*   Подсказка 2: а еще вам понадобится `gensim.models.Word2Vec` (см. выше)
*   Подсказка 3: без остального в принципе можно обойтись. Но если подать тексты сырыми, то модель обучится на токенах. "Кот", "кот" и "кота" будут для нее разными словами. Зато "печь" будет одним словом вне завимисимости от части речи. Поэтому можно еще воспользоваться функцией `tag_ud` выше и обучить модель как у русвекторес. Но осторожно. UDPipe-ом обработать всю Войну и Мир — это минут 6-7.

Без предобработки ваша модель сможет как-то вот так: 

<img src = "https://github.com/dhhse/dh2020/raw/master/pics/napoleon_token.png">
<img src = "https://github.com/dhhse/dh2020/raw/master/pics/war_token.png">

C предобработкой будет как в моделях русвекторес: лемма с пос-тегом. И это на русском работает осмысленнее обычно: 

<img src = "https://github.com/dhhse/dh2020/raw/master/pics/war_lemma_pos.png">

In [ ]:
!wget 'https://github.com/dhhse/dh2020/raw/master/data/wap_w2v.txt' 

--2021-05-13 15:53:10--  https://github.com/dhhse/dh2020/raw/master/data/wap_w2v.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhhse/dh2020/master/data/wap_w2v.txt [following]
--2021-05-13 15:53:10--  https://raw.githubusercontent.com/dhhse/dh2020/master/data/wap_w2v.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5171830 (4.9M) [text/plain]
Saving to: ‘wap_w2v.txt’

wap_w2v.txt         100%[===================>]   4.93M  25.6MB/s    in 0.2s    

2021-05-13 15:53:11 (25.6 MB/s) - ‘wap_w2v.txt’ saved [5171830/5171830]



In [ ]:
wap = 'wap_w2v.txt'
data = gensim.models.word2vec.LineSentence(wap)

In [ ]:
model_wap = gensim.models.Word2Vec(data, size=500, window=10, min_count=2, sg=0)

2021-05-13 15:54:45,536 : INFO : collecting all words and their counts
2021-05-13 15:54:45,539 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-05-13 15:54:45,628 : INFO : PROGRESS: at sentence #10000, processed 161603 words, keeping 39106 word types
2021-05-13 15:54:45,717 : INFO : PROGRESS: at sentence #20000, processed 335781 words, keeping 65326 word types
2021-05-13 15:54:45,786 : INFO : collected 81592 word types from a corpus of 462159 raw words and 27123 sentences
2021-05-13 15:54:45,789 : INFO : Loading a fresh vocabulary
2021-05-13 15:54:45,864 : INFO : effective_min_count=2 retains 30236 unique words (37% of original 81592, drops 51356)
2021-05-13 15:54:45,866 : INFO : effective_min_count=2 leaves 410803 word corpus (88% of original 462159, drops 51356)
2021-05-13 15:54:45,959 : INFO : deleting the raw counts dictionary of 81592 items
2021-05-13 15:54:45,964 : INFO : sample=0.001 downsamples 36 most-common words
2021-05-13 15:54:45,966 : INFO :

In [ ]:
model_wap.most_similar('Наташа')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('долго', 0.9984106421470642),
 ('Соню,', 0.9982855319976807),
 ('что-то', 0.9982360601425171),
 ('нее,', 0.9981175065040588),
 ('за', 0.9978007078170776),
 ('Наташе,', 0.9977572560310364),
 ('грудь', 0.997657299041748),
 ('заметила,', 0.9976304769515991),
 ('слушал', 0.9974093437194824),
 ('где', 0.9973325729370117)]

In [ ]:
len(model_wap.wv.vocab)

30236

In [ ]:
text = open('wap_w2v.txt', 'r', encoding='utf-8').read()
processed_text = tag_ud(text=text, modelfile=modelfile)
print(processed_text[:350])
with open('wap_w2v_processed.txt', 'w', encoding='utf-8') as out:
    out.write(processed_text)


Loading the model...
Processing input...


то_PRON первый_ADJ

часть_NOUN первый_ADJ

i_NUM

Eh_PROPN bien_X mon_X prince_X
Gênes_PROPN et_X Lucques_PROPN ne_X sont_X plus_X que_X des_X apanages_X des_X поместье_NOUN de_X la_X famille_X Buonaparte_PROPN
Non_PROPN je_X vous_X préviens_X que_X si_X vous_X ne_X me_X dites_X pas_X que_X nous_X avons_X la_X guerre_X si_X vous_X vous_X permettez_


In [ ]:
data = gensim.models.word2vec.LineSentence('wap_w2v_processed.txt')

In [ ]:
model_wap_lemmas_pos = gensim.models.Word2Vec(data, size=300, window=10, min_count=2, sg=0)

2021-05-13 16:07:24,984 : INFO : collecting all words and their counts
2021-05-13 16:07:24,987 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-05-13 16:07:25,076 : INFO : PROGRESS: at sentence #10000, processed 156252 words, keeping 15798 word types
2021-05-13 16:07:25,181 : INFO : PROGRESS: at sentence #20000, processed 324927 words, keeping 24155 word types
2021-05-13 16:07:25,263 : INFO : collected 29037 word types from a corpus of 447977 raw words and 27064 sentences
2021-05-13 16:07:25,269 : INFO : Loading a fresh vocabulary
2021-05-13 16:07:25,300 : INFO : effective_min_count=2 retains 13506 unique words (46% of original 29037, drops 15531)
2021-05-13 16:07:25,301 : INFO : effective_min_count=2 leaves 432446 word corpus (96% of original 447977, drops 15531)
2021-05-13 16:07:25,343 : INFO : deleting the raw counts dictionary of 29037 items
2021-05-13 16:07:25,346 : INFO : sample=0.001 downsamples 50 most-common words
2021-05-13 16:07:25,348 : INFO :

In [ ]:
model_wap_lemmas_pos.most_similar ('Наташа_PROPN')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Пьер_PROPN', 0.9753262996673584),
 ('Марья_PROPN', 0.9669135808944702),
 ('осведомляться_VERB', 0.9598015546798706),
 ('удивление_NOUN', 0.9585548639297485),
 ('письмо_NOUN', 0.9561311602592468),
 ('отъезд_NOUN', 0.9557530283927917),
 ('знакомый_NOUN', 0.9538062810897827),
 ('холодность_NOUN', 0.9526064395904541),
 ('разговор_NOUN', 0.9524383544921875),
 ('смутный_ADJ', 0.9505127668380737)]

In [ ]:
len(model_wap_lemmas_pos.wv.vocab)

13506


## FastText: эмбеддинги n-граммов

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грамов. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

### Для работы с fasttext-моделью придется обновить gensim

Корректная работа с fasttext-моделями гарантируется от версии 3.7.2, а в колабе по умолчанию генсим 3.6.0

In [97]:
import gensim
gensim.__version__

'4.1.2'

In [98]:
!pip install --upgrade gensim

You should consider upgrading via the '/Users/Alexey.Zhebel/IdeaProjects/CompLing/venv/bin/python -m pip install --upgrade pip' command.


После этого надо перезапустить среду (колаб и сам вам предложит это сделать)

In [99]:
import gensim

In [100]:
gensim.__version__

'4.1.2'

### Отлично, теперь возьмем русскую фасттекст-модель 
из уже известной нам [коллекции](https://rusvectores.org/ru/models/) RusVectores

In [101]:
!wget 'http://vectors.nlpl.eu/repository/20/214.zip'

--2022-03-19 11:25:58--  http://vectors.nlpl.eu/repository/20/214.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1920218982 (1.8G) [application/zip]
Saving to: ‘214.zip’

214.zip             100%[===================>]   1.79G  7.25MB/s    in 4m 29s  

2022-03-19 11:30:28 (6.81 MB/s) - ‘214.zip’ saved [1920218982/1920218982]



В генсим её надо загружать чуть иначе, чем word2vec-овскую модель. Надо сначала распаковать архив:

In [102]:
!unzip '214.zip'

Archive:  214.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [103]:
fasttext_model = gensim.models.KeyedVectors.load('model.model')

2022-03-19 11:30:49,920 : INFO : loading KeyedVectors object from model.model
2022-03-19 11:30:51,327 : INFO : loading vectors from model.model.vectors.npy with mmap=None
2022-03-19 11:30:51,559 : INFO : loading vectors_vocab from model.model.vectors_vocab.npy with mmap=None
2022-03-19 11:30:51,768 : INFO : loading vectors_ngrams from model.model.vectors_ngrams.npy with mmap=None
2022-03-19 11:30:52,395 : INFO : setting ignored attribute vectors_norm to None
2022-03-19 11:30:52,396 : INFO : setting ignored attribute vectors_vocab_norm to None
2022-03-19 11:30:52,396 : INFO : setting ignored attribute vectors_ngrams_norm to None
2022-03-19 11:30:52,397 : INFO : setting ignored attribute buckets_word to None
2022-03-19 11:30:58,580 : INFO : FastTextKeyedVectors lifecycle event {'fname': 'model.model', 'datetime': '2022-03-19T11:30:58.580209', 'gensim': '4.1.2', 'python': '3.8.9 (default, Oct 26 2021, 07:25:54) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.1-x86_64-i386-64

### косинусная близость на примерах

In [110]:
fasttext_model.most_similar("экспрессо")

[('экспрессом', 0.82410728931427),
 ('экспрессов', 0.8085681796073914),
 ('экспресса', 0.7968765497207642),
 ('экспрессы', 0.7598093748092651),
 ('экспрессе', 0.7217636108398438),
 ('экспрессии', 0.6977290511131287),
 ('экспрессия', 0.688822329044342),
 ('экспресс', 0.6845555901527405),
 ('экспрессию', 0.6839092373847961),
 ('экспрессией', 0.6713719964027405)]

In [107]:
fasttext_model.most_similar ('кравать') # попробовать разные опечатки

[('вать', 0.7385567426681519),
 ('хавать', 0.7380610704421997),
 ('быковать', 0.7299599647521973),
 ('воровать', 0.721983790397644),
 ('пировать', 0.7135821580886841),
 ('спаивать', 0.6996143460273743),
 ('жировать', 0.6917222738265991),
 ('кать', 0.6883906126022339),
 ('стращать', 0.6878533363342285),
 ('бать', 0.6829023361206055)]

In [106]:
fasttext_model.most_similar ('блять')

[('бля', 0.7755434513092041),
 ('блеать', 0.7372725009918213),
 ('мля', 0.708987295627594),
 ('нахуй', 0.691758394241333),
 ('ваще', 0.6896630525588989),
 ('блин', 0.6857178211212158),
 ('ебаные', 0.675656795501709),
 ('ебаный', 0.6753869652748108),
 ('охуеть', 0.6736325621604919),
 ('сука', 0.6635270118713379)]

In [108]:
fasttext_model.most_similar ('котэ')

[('кот', 0.6769469380378723),
 ('котя', 0.6286473274230957),
 ('котик', 0.6228466033935547),
 ('котяра', 0.6106880307197571),
 ('кота', 0.6015534996986389),
 ('коты', 0.5914632678031921),
 ('котика', 0.5902719497680664),
 ('бегемотик', 0.5805137157440186),
 ('котики', 0.5798563957214355),
 ('котенок', 0.5771116018295288)]

In [109]:
fasttext_model.most_similar ('некузявый')

[('корявый', 0.7026479244232178),
 ('чернявый', 0.6892554759979248),
 ('писклявый', 0.6638097167015076),
 ('некрасивый', 0.6533088088035583),
 ('бестолковый', 0.6427179574966431),
 ('сопливый', 0.6411333680152893),
 ('хреновый', 0.6380565762519836),
 ('паршивый', 0.6377483606338501),
 ('махонький', 0.6357012987136841),
 ('прыщавый', 0.6353145837783813)]

In [111]:
fasttext_model.most_similar ('лебедиво')

[('лебеди', 0.692702054977417),
 ('лебедин', 0.6209226250648499),
 ('диво', 0.6060458421707153),
 ('лебединое', 0.6034351587295532),
 ('лебеду', 0.588827908039093),
 ('лебеда', 0.5634397268295288),
 ('лебединой', 0.5597955584526062),
 ('лебедь', 0.5428179502487183),
 ('пеликаны', 0.5409451127052307),
 ('лебединая', 0.5382550358772278)]

In [112]:
fasttext_model.most_similar ('lol') # латиницу русская модель не делает, делает веселое

[(':d', 0.682301938533783),
 ('quote', 0.6119906902313232),
 ('лол', 0.5925002694129944),
 ('ахахаха', 0.5776768326759338),
 ('facepalm', 0.576578676700592),
 ('ахаха', 0.5749382972717285),
 ('cry', 0.573516845703125),
 ('biggrin', 0.5696468353271484),
 ('p.s', 0.5673760175704956),
 ('wink', 0.5664108395576477)]

## Как выйти за пределы слов и применить это на практике?

Как я уже говорил, векторные модели сделали не для того, чтобы веселиться с векторной математикой (ну или не только для этого). В первую очередь это супер-полезный способ **векторизации текста** для практических задач комплингвистики:


*   Классификация текстов
*   Извлечение информации (которое часто сводится к задаче классификации слов или их последовательностей)
*   Анализ тональности
*   И прочее

Если объектом является не отдельное слово, а предложение или текст (так бывает часто, например, при решении все той же задачи классификации текстов) то общая идея такая: весь текст превращается в вектор (набор чиселок), которые как-то зависят от векторов его слов (множества наборов чиселок). 

Самый очевидный вариант — сложить вектора всех слов текста или взять средний вектор всех слов текста.



In [113]:
import numpy as np

In [114]:
fasttext_model.vectors.shape

(347295, 300)

In [116]:
np.average(np.array([fasttext_model['глокая'], fasttext_model['куздра'], fasttext_model['штеко']]))

0.005346726

Чуть более тонкое есть в алгоритме doc2vec — его сделали те же люди, что и word2vec. Они придумали как бы добавлять еще одно псевдо-слово в контекст при обучении векторов слов для word2vec. В результате у нас после обучения кроме векторов слов есть еще один вектор той же размерности, который как бы побывал в контексте всех слов данного документа. Он и выдается в качестве вектора (эмбеддинга) документа. 

<img src = "https://miro.medium.com/max/535/0*x-gtU4UlO8FAsRvL.">

Реализация `doc2vec`



# Заключение

В этом тьюториале мы постарались разобраться с тем, как работать с семантическими векторными моделями и библиотекой **gensim**. Теперь вы можете:
* использовать готовые модели векторной семантики,осуществлять простые операции над векторами слов.
* осуществлять предобработку текстовых данных, что может пригодиться во многих задачах обработки естественного языка;
* тренировать векторные семантические модели. Формат моделей совместим с моделями, представленными на веб-сервисе **RusVectōrēs**;



## Что мы не затронули?



*   Оценка качества моделей (как тут считать точность-полноту-F-меру). См например в [этой тетрадке](https://github.com/ancatmara/data-science-nlp/blob/master/2.%20Embeddings.ipynb) у Оксаны оценку качества на задачах оценки семантической близости и поиска аналогии (Москва Россия Берлин Германия).
*   Применение для реальных задач классификации текстов. См. в [этой тетрадке](https://github.com/mannefedov/compling_nlp_hse_course/blob/master/2020/Embeddings.ipynb) Миши Нефедова.

